## Análise de Tiroteios no RJ
### Trabalho da Pós em Arquitetura de Software 

Este projeto utiliza dados públicos da API do Fogo Cruzado para analisar padrões de tiroteios na cidade do Rio de Janeiro.

### Objetivo

Identificar se há mais risco de eventos com vítimas fatais em determinados horários e dias da semana.


In [ ]:
#Importação das bibliotecas necessárias
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from pickle import dump
from pickle import load

In [2]:
def periodo_do_dia(h):
    """
    Recebe um horário em formato de 24 horas (0-23) e retorna o período do dia:
    Arguments:
    h -- Hora no formato 0-23
    Returns:
    0 -- madrugada (00:00-05:59)
    1 -- manhã (06:00-11:59)
    2-- tarde (12:00-17:59)
    3-- noite (18:00-23:59)
    """
    if h < 6: return 0 # madrugada
    elif h < 12: return 1 # manha
    elif h < 18: return 2 # tarde
    else: return 3 # noite

def quantidade_vitimas_mortas(vitimas):
    """ Conta o número de vítimas mortas em uma lista de vítimas."""
    return contar_por_status(vitimas, 'Dead')

def parse_vitimas(v):
        """Converte uma string ou dicionário de vítimas em uma lista de vítimas."""
        if isinstance(v, str) and v.strip().startswith('['):
            try:
                return ast.literal_eval(v)
            except (ValueError, SyntaxError):
                return []
        return v if isinstance(v, list) else []

def parse_name(loc):
    """Extrai o nome de uma localização a partir de uma string ou dicionário. Serve para os campos neighbohhod e locality"""
    if(loc == 'nan'):
         return 'Desconhecido'
    if isinstance(loc, str) and loc.strip().startswith('{'):
            try:
                lit =  ast.literal_eval(loc)
                return lit.get('name', 'Desconhecido')
            except (ValueError, SyntaxError):
                return 'Desconhecido'
    if isinstance(loc, dict):
        return loc.get('name', 'Desconhecido')
    return 'Desconhecido'

def contar_por_status(vitimas, status):
    """Conta o número de vítimas com um determinado status em uma lista de vítimas."""
    if not isinstance(vitimas, list):
        return 0
    return sum(1 for vit in vitimas if vit.get('situation') == status)

Para esse caso de uso vamos usar as colunas de vítimas, data e hora se houve ação policial e se tem algum agente envolvido.

Para esse trabalho estamos desconsiderando as colunas abaixo:
<br>
'id', 'documentNumber', 'state', 'region', 'city', 'animalVictims', 'contextInfo', 'relatedRecord', 'subNeighborhood'

In [23]:
#Leitura dos dados Atualmente marcados até a data 
url = "fogo-cruzado/tiroteios_RJ.csv" # substituir pela URL no github
colunas = ['address', 'neighborhood', 'locality', 'latitude', 'longitude', 'policeAction', 'date', 'agentPresence', 'victims']
dataset = pd.read_csv(url, usecols=colunas, skiprows=0, delimiter=',', encoding='utf-8-sig')
# dataset.head()


In [29]:
# Classificando os dados

#tratando os dados de vitimas
dataset['victims'] = dataset['victims'].apply(parse_vitimas)
dataset['total_victims'] = dataset['victims'].apply(lambda v: len(v) if isinstance(v, list) else 0)
dataset['deaths'] = dataset['victims'].apply(quantidade_vitimas_mortas)

#Tratando os dados temporais, vamos recuperar as frequenciais semanais e mensais e o periodo do dia em que o tiroteio ocorreu
dataset['date'] = pd.to_datetime(dataset['date'])
dataset['weekday'] = dataset['date'].dt.day_name()
dataset['month'] = dataset['date'].dt.month
dataset['period'] = dataset['date'].dt.hour.apply(periodo_do_dia)

#tratando os dados de localização que estavam em formato de dicionário, recuperando somente o nome
dataset['neighborhood'] = dataset['neighborhood'].apply(parse_name)
dataset['locality'] = dataset['locality'].apply(parse_name)
dataset.drop(['victims', 'date'], axis=1, inplace=True)
dataset.head()


,address,neighborhood,locality,latitude,longitude,policeAction,agentPresence,total_vitimas,total_mortas,weekday,month,period,total_victims,deaths
0,"Cidade de Deus, Rio de Janeiro - RJ, Brasil",Desconhecido,Desconhecido,-22.948170,-43.362671,True,True,0,0,Tuesday,7,0,0,0
1,"Nova Brasília - Complexo do Alemão, Rio de Jan...",Desconhecido,Desconhecido,-22.866065,-43.272438,True,True,0,0,Tuesday,7,0,0,0
2,"Rua do Engenho Novo - Sampaio, Rio de Janeiro ...",Desconhecido,Desconhecido,-22.901310,-43.263218,False,False,0,0,Tuesday,7,0,0,0
3,"Favela do Rola - Santa Cruz, Rio de Janeiro - ...",Desconhecido,Desconhecido,-22.919651,-43.684387,True,True,1,1,Tuesday,7,1,1,1
4,"Acari, Rio de Janeiro - State of Rio de Janeir...",Desconhecido,Desconhecido,-22.825531,-43.344460,True,True,0,0,Tuesday,7,1,0,0
